# Usage examples 

In [ ]:
# this cell is hidden in the docs
# it installs the requirements to run this demo
!python -m pip install qutree[demo]

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/alice4space/qutree/blob/main/docs/source/examples/demo.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20colab-blue?logo=googlecolab&labelColor=555)](https://colab.research.google.com/github/alice4space/qutree/blob/main/docs/source/examples/demo.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20mybinder-orange?logo=jupyter&labelColor=555)](https://mybinder.org/v2/gh/alice4space/qutree/main?labpath=docs%2Fsource%2Fexamples%2Fdemo.ipynb)

In [ ]:
from urllib.request import urlretrieve
import tempfile

import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from scipy.stats import unitary_group 

In [ ]:
from qutree import BBT

## Bell state preparation

In [ ]:
@qml.qnode(qml.device("default.qubit", wires=2))
def bell(t):
    qml.Hadamard(wires=[0])
    qml.CRY(np.pi*t,wires=[0,1])
    return qml.state()

ts = np.linspace(0,1,21)
Ss = np.array([bell(t) for t in ts]).T

In [ ]:
bbt = BBT(2)
bbt.add_data(Ss)
bbt.plot_tree()

The top sphere shows the effect of the hadamard gate : an even superposition between the two basis state without phase offset. 
Then on the bottom sphere we wee the effect of the parametrised Control Y rotattion :
The left sphere represent the subspace where the first qubit is 0. it is of course untouched by the CY gate.
The right sphere represent the subspace where the first qubit is 1. The state is gradually rotated about the Y axis from 0 (blue) to 1 (red).
The bell state corresponds to the red point.

## Study how an encoding spans the hilbert space

In [ ]:
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def circuit(t):
    state = np.array([1, 2j, 3, t*1j, 5, 6j, 7, 8j])
    state = state / np.linalg.norm(state)
    qml.MottonenStatePreparation(state_vector=state, wires=range(3))
    return qml.state()

print(qml.draw(circuit, expansion_strategy="device", max_length=80)(4))

In [ ]:
ts = np.linspace(0,8,21)
Ss = np.array([circuit(t) for t in ts]).T

In [ ]:
bbt = BBT(3)
bbt.add_data(Ss)
bbt.plot_tree()

## quantum states dataset from Machine Learning

The data is generated using the demo from pennylane about classification that can be found here : https://pennylane.ai/qml/demos/tutorial_variational_classifier.html

In [ ]:
# read the file from data folder in github
with tempfile.NamedTemporaryFile() as tmp:
    url = 'https://raw.githubusercontent.com/alice4space/qutree/main/docs/source/examples/data/iris_quantum_kernel.npy'
    urlretrieve(url, tmp.name)
    states_ml = np.load(tmp.name,allow_pickle=True)

In [ ]:
bbt = BBT(4)
lam = 0.3
cs = np.concatenate([np.random.random(50)*lam,np.random.random(50)*lam+1-lam])
bbt.add_data(states_ml.T,colors=cs)
bbt.plot_tree()

## hamiltonian simulation

In [ ]:
num_qubits = 4
U = unitary_group.rvs(2**num_qubits,random_state = 0)
D,S = np.linalg.eigh(U)

### excite only two eigenstates : a single frequency

In [ ]:
ka = 12
kb = 2

S0 = (S[:,ka]+S[:,kb])/np.sqrt(2)
Sts = []
ts = np.linspace(0,2*np.pi/np.abs(D[ka]-D[kb]),101)
for t in ts:
    Sts.append(S @ np.diag(np.exp(1j*D*t)) @ S.T.conj() @ S0)
Sts = np.array(Sts).T

In [ ]:
bbt = BBT(4)
bbt.add_data(Sts,cmap='hsv')
bbt.plot_tree()

### excite three eigenstates : three frequencies

In [ ]:
ka = 12
kb = 2
kc = 4

S0 = (S[:,ka]+S[:,kb]+S[:,kc])/np.sqrt(3)
Sts = []
ts = np.linspace(0,5,101)
for t in ts:
    Sts.append(S @ np.diag(np.exp(1j*D*t)) @ S.T.conj() @ S0)
Sts = np.array(Sts).T

In [ ]:
bbt = BBT(4)
bbt.add_data(Sts,cmap='jet')
bbt.plot_tree()